In [12]:
import obspy
from obspy import read
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
client = Client('IRIS')

import matplotlib.pyplot as plt
import numpy as np
import os

In [13]:
#Make folder for cleaned data
! mkdir data/clean
filepath = os.getcwd() + '/data/clean/'
raw_filepath = os.getcwd() + '/data/raw/'

mkdir: data/clean: File exists


### Cleaning/Processing Steps
The data will be demeaned similar to Zali et al., using a more conservative downsample rate of 20 Hz, given volcanic tremor is typically between 1-9 Hz.

*Read in mseed files <br>
*Check all are the same length <br>
*Demean and detrend <br>
*Anti-alias filter and downsample data to 20 Hz

In [14]:
#demean, detrend, lowpass filter, and downsample

def process(st, buffer, freq, max_target_frequency): #freq is the original sampling frequency
    tr = st[0].copy()

    #demean and detrend
    tr.detrend('demean')
    tr.detrend('linear')

    #taper data (to specified taper buffer)
    tr.taper(max_percentage=0.05)

    #lowpass filter data to avoid aliasing when downsampling
    tr.filter('lowpass', freq=2*max_target_frequency, zerophase=True)

    #downsample to 20 Hz
    dec_factor = int(freq / (2*max_target_frequency))
    tr.decimate(dec_factor, no_filter=True)

    #trim off tapered edges
    tr.trim(starttime=tr.stats.starttime+buffer, endtime=tr.stats.endtime-buffer)

    return(tr)

In [15]:
#read in data, process, save cleaned data and figures
length = np.array([])

for i in range(100):
    i += 1
    mseed_name = str(i)+'_sitkin.mseed'
    st = read(raw_filepath+mseed_name)

    #parameters for processing function
    buffer = 60*60*24*0.05 #seconds
    freq = st[0].stats.sampling_rate
    max_target_freq = 10

    #do processing
    tr = process(st, buffer, freq, max_target_freq)

    #save trace to mseed again
    tr.write(filepath+str(i)+'_cleaned.mseed', format='MSEED')

    #add data length to array
    length = np.append(length, len(tr.data))

    #plot and save figures
    fig, ax = plt.subplots()
    fig.set_size_inches(6,6)
    ax.plot(tr.times('relative'), tr.data, c='k')
    ax.set_title('Data for Day '+mseed_name.split('_')[0])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Counts')
    ax.set_xlim(0, len(tr.data)/freq)
    ax.grid()
    plt.tight_layout()
    plt.savefig(filepath+'Day'+mseed_name.split('_')[0]+'.png')
    plt.close();

Save simple time series plots to visually inspect for outliers/data issues not a simple way to remove outliers algorithmically, day 87 for example has extremely impulsive events that look like malfunctions, but are real

In [16]:
#check that all values in length are equal
np.unique(length)

array([2121346., 2159880., 2160001.])

This is the correct number of points (+1) for 24 hours of data sampled at 25 Hz. The single extra point is a product of the decimation, will be removed in the preparing ai ready data notebook.